In [10]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *

from epsampling.utils import load_csv

# pd.set_option('display.float_format', lambda x: '%.3f' % x)

data_dir = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'

In [11]:
from epsampling.utils import load_csv

data_dir = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'
# timestamp = '20240925-112125'

# 20240925-132841
df_deaths,_ = load_csv('df_deaths_pastweeks', data_dir=f'{data_dir}cached/', timestamp='20240925-132841')

## Use acs feats that have alrdy been filtered by correlation ...
# df_acs,_ = load_csv('df_acs', data_dir=f'{data_dir}cached/', timestamp=timestamp)
df_acs,_ = load_csv('df_acs_filt', data_dir=f'{data_dir}cached/', timestamp='20240925-113849')
# df_acs.set_index('Fips',inplace=True,drop=True)

# display(df_deaths, df_acs)

In [12]:
## Get rid of bad dates.
# df_deaths = df_deaths.replace({np.nan: -1})
df_deaths.dropna(inplace=True)

In [13]:
df = df_deaths.merge(df_acs,on='Fips')
# df.State_fips = df.State_fips.astype(str)
# df.Fips = df.Fips.astype(str)
# df.dtypes

In [14]:
dates = sorted(df.Date.unique())
len(dates)

99

In [18]:
# # # # # # # # # # # # 
test_size = 0.05
# # # # # # # # # # # # 

# # # # # # # # # # # # # # # # # # # # # # # #
# # # # # Choose split  # # # # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # # # # #

## Date split (extrapolate)
dates = sorted(df.Date.unique())
train_size = 1.0 - test_size
lim = int(len(dates) * train_size)
dates_train = dates[:lim]
dates_test = dates[lim:]
df_train = df[df.Date.isin(dates_train)]
df_test = df[df.Date.isin(dates_test)]

## Random split
# from sklearn.model_selection import train_test_split
# df_train, df_test = train_test_split(df, test_size=test_size, random_state=666)

# # # # # # # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # # # # #

# death_covs = ['COVIDhubEns_state_deaths', 'Pop', 'Pop_ratio', 'Naive_county_deaths', 'Timestring']

death_covs = ['State_fips', 'Fips', 
              'COVIDhubEns_state_deaths', 'Pop', 'Pop_ratio', 
       'Naive_county_deaths', 'Timestring', 'Weeks_ago_4', 'Weeks_ago_5',
       'Weeks_ago_6', 'Weeks_ago_7', 'Weeks_ago_8']
feats = death_covs + df_acs.columns.values.tolist()[1:]

X_train = df_train[feats]
X_test = df_test[feats]

y_train = df_train['True_county_deaths']
y_test = df_test['True_county_deaths']
y_naive = df_test['Naive_county_deaths']

print(len(X_train), len(X_test))

291683 15645


In [19]:
from sklearn.ensemble import RandomForestRegressor
# train time for random split at 0.1 test_size? 2 min
clf = RandomForestRegressor(random_state=666, max_depth=6)
clf.fit(X_train, y_train)

RandomForestRegressor(max_depth=6, random_state=666)

In [20]:
from epsampling.utils import get_performance

y_pred = clf.predict(X_test)

model_names = ['Deaths_pred', 'Deaths_naive']
model_preds = [y_pred, y_naive]

metrics_dict = get_performance(model_names, model_preds, y_test, y_naive)
metrics_dict

{'Deaths_pred': {'MAE': 16.49091376565517,
  'MSE': 3262.1729600849417,
  'r2': 0.9964522685299836,
  'relMAE': 2.6897388332171364},
 'Deaths_naive': {'MAE': 76.79841473195529,
  'MSE': 66177.0495098121,
  'r2': 0.9280300572619902,
  'relMAE': 1.0}}

{
'Time_split': 
 {'MAE': 16.49091376565517,
  'MSE': 3262.1729600849417,
  'r2': 0.9964522685299836,
  'relMAE': 2.6897388332171364},

'Random_split':
 {'MAE': 10.500567348407527,
  'MSE': 1197.426512426188,
  'r2': 0.9970360255035902,
  'relMAE': 2.654721662037693},

'Deaths_naive': 
 {'MAE': 50.15511940054299,
  'MSE': 37201.74637541338,
  'r2': 0.9180466593308316,
  'relMAE': 1.0}
}